In [1]:
import os
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import time
from bs4 import BeautifulSoup
import pandas as pd

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Configurações do usuário
EMAIL = 'leonardo.campos@rigarr.com.br'
SENHA = 'Br@sil32aaaaaaa'
URL_LOGIN = "https://one.bees.com/"
DOWNLOADS_PATH = 'C:\\Users\\leonardo.campos\\Documents\\Automacao'  # Certifique-se de que este caminho está correto
EXCEL_PATH = 'G:\\Drives compartilhados\\EQUIPE DE VENDAS SP\\TABELA DE PREÇOS SP\\TABELA DE PREÇO SP 2.0.xlsx'

def inicializa_driver():
    return webdriver.Chrome()

def aceitar_cookies(driver):
    try:
        WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
        ).click()
    except NoSuchElementException:
        print("Botão de aceitação de cookies não encontrado.")

def fazer_login(driver):
    try:
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="signInName"]'))).send_keys(EMAIL)
        driver.find_element(By.XPATH, '//*[@id="next"]').click()
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="password"]'))).send_keys(SENHA)
        driver.find_element(By.XPATH, '//*[@id="next"]').click() 
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="single-spa-application:@admin-portal/main-mfe"]/section/header/div[1]/div/div[1]/div[3]/div[2]/button'))).click()

    except TimeoutException:
        print("Tempo de espera expirado ao tentar realizar login.")
    except NoSuchElementException as e:
        print(f"Elemento não encontrado durante o login: {e}")
    except Exception as e:
        print(f"Erro durante o login: {e}")

def acessar_pagina_inventario(driver):
    try:
        driver.get("https://one.bees.com/inventory-admin")
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="single-spa-application:@admin-portal/main-mfe"]/section/header/div[1]/div/div[1]/div[3]/div[2]/button'))).click()

        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="some-identifier-on-inventory-page"]'))  # Atualize com um identificador real
        )
        print("Página de inventário acessada com sucesso.")
    except TimeoutException:
        print("Timeout ao tentar acessar a página de inventário.")
    except NoSuchElementException:
        print("Elemento da página de inventário não encontrado.")
    except Exception as e:
        print(f"Erro ao acessar a página de inventário: {e}")

def raspar_dados(driver):
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        dados = soup.find_all('p')
        saldos = soup.find_all('span')[8:]  # Ajuste o índice conforme necessário

        codigos = []
        descricoes = []
        saldos_estoque = []
        imagens_urls = []

        for idx, elemento in enumerate(dados):
            texto = elemento.text.strip()
            if idx > 0:  # Ignorar o primeiro elemento se não for necessário
                if idx % 2 == 1:
                    descricoes.append(texto)  # Descrição
                else:
                    codigos.append(texto)  # Código

        for saldo in saldos:
            saldos_estoque.append(saldo.text.strip())

        # Buscando imagens
        imagens = soup.find_all('img')  # Supondo que as imagens estão em tags <img>
        for img in imagens:
            img_url = img['src'] if 'src' in img.attrs else ''
            imagens_urls.append(img_url)

        tabela = []
        for i in range(min(len(descricoes), len(codigos), len(saldos_estoque), len(imagens_urls))):
            tabela.append({
                'Código': codigos[i],
                'Descrição': descricoes[i],
                'Saldo': saldos_estoque[i],
                'Imagem URL': imagens_urls[i],  # Adiciona URL da imagem
                'Código Filial': ''  # Coluna para código filial em branco
            })

        df = pd.DataFrame(tabela)
        print(f"Dados raspados: {df.shape[0]} linhas")
        return df
    except Exception as e:
        print(f"Erro ao raspar dados: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro


def clicar_proximo(driver):
    try:
        proximo_btn = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="radix-0-content-inventory"]/div/div[2]/button[2]'))
        )
        if 'disabled' not in proximo_btn.get_attribute('class'):
            proximo_btn.click()
            time.sleep(10)  # Aguarde um pouco para a próxima página carregar
            return True
        else:
            return False
    except (ElementClickInterceptedException, NoSuchElementException, TimeoutException):
        print("Indo para a próxima filial...")
        return False

def selecionar_filial(driver, nome_filial):
    try:
        WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="single-spa-application:@bees-hub/inventory-admin-mfe"]/div/div/div[1]/div/div[2]/div/div[2]/div/button'))
        ).click()
        
        time.sleep(3)  # Espera adicional para garantir que o menu esteja carregado
        
        filial_element = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, f'//span[text()="{nome_filial}"]'))
        )
        filial_element.click()
        
        print(f"Filial '{nome_filial}' selecionada com sucesso.")
        time.sleep(5)  # Aguarde um pouco para garantir que a página da filial esteja carregada
    except NoSuchElementException:
        print("Elemento para seleção de filial não encontrado.")
    except TimeoutException:
        print("Timeout ao tentar selecionar a filial.")
    except Exception as e:
        print(f"Erro ao selecionar a filial: {e}")

def ir_para_primeira_pagina(driver):
    try:
        while True:
            botao_primeiro = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="radix-0-content-inventory"]/div/div[2]/button[1]'))
            )
            if 'disabled' in botao_primeiro.get_attribute('class'):
                break
            botao_primeiro.click()
            time.sleep(3)  # Tempo para garantir que a página esteja atualizada

        print("Navegação para a primeira página realizada com sucesso.")
    except NoSuchElementException:
        print("Botão de primeira página não encontrado.")
    except TimeoutException:
        print("Timeout ao tentar ir para a primeira página.")
    except Exception as e:
        print(f"Erro ao tentar ir para a primeira página: {e}")

def raspagem_por_filial(driver, nome_filial):
    selecionar_filial(driver, nome_filial)
    ir_para_primeira_pagina(driver)  # Garantir que esteja na primeira página

    dfs = []
    while True:
        df = raspar_dados(driver)
        if df.empty:
            print(f"Nenhum dado encontrado na página atual da filial '{nome_filial}'.")
            break
        dfs.append(df)
        if not clicar_proximo(driver):
            break

    if dfs:
        final_df = pd.concat(dfs, ignore_index=True)
        final_df['Código Filial'] = nome_filial  # Preencher a coluna Código Filial
        output_path = os.path.join(DOWNLOADS_PATH, f'inventario_filial_{nome_filial}.csv')
        final_df.to_csv(output_path, index=False)
        print(f"Dados da filial '{nome_filial}' salvos em '{output_path}'.")
        return output_path
    else:
        print(f"Nenhum DataFrame foi criado para a filial '{nome_filial}'.")
        return None

def comparar_dados(inventario_path, excel_path):
    try:
        df_inventario = pd.read_csv(inventario_path)
        df_precos = pd.read_excel(excel_path)

        # Garantir que as colunas estejam em minúsculas para evitar problemas com capitalização
        df_inventario.columns = df_inventario.columns.str.lower()
        df_precos.columns = df_precos.columns.str.lower()

        if 'descrição' in df_inventario.columns and 'produto' in df_precos.columns:
            for index, row in df_inventario.iterrows():
                descricao = row['descrição']
                # Se a descrição for encontrada nos preços, atualiza o saldo
                if descricao in df_precos['produto'].values:
                    df_inventario.at[index, 'Código Filial'] = row['Código Filial'] or 'Novo Código'
                else:
                    df_inventario.at[index, 'Código Filial'] = 'Novo Código'  # ou deixar em branco se preferir

            # Salvar o resultado em um novo arquivo CSV
            output_csv_path = inventario_path.replace('.csv', '_resultado_comparacao.csv')
            df_inventario.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
            print(f"Comparação concluída. Resultados salvos em '{output_csv_path}'")
        else:
            print("Colunas 'Descrição' e/ou 'Produto' não encontradas nos arquivos.")
    except Exception as e:
        print(f"Erro ao comparar dados: {e}")

def main():
    driver = inicializa_driver()
    driver.get(URL_LOGIN)
    try:
        aceitar_cookies(driver)
        fazer_login(driver)
        acessar_pagina_inventario(driver)

        # Lista de filiais a serem raspadas
        filiais = ['RIGARRSPCAPITAL', 'RIGARRRJCAPITAL', 'RIGARRMGCAPITAL']
        for filial in filiais:
            print(f"Iniciando raspagem para a filial: {filial}")
            inventario_path = raspagem_por_filial(driver, filial)
            if inventario_path:
                comparar_dados(inventario_path, EXCEL_PATH)
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

Timeout ao tentar acessar a página de inventário.
Iniciando raspagem para a filial: RIGARRSPCAPITAL
Filial 'RIGARRSPCAPITAL' selecionada com sucesso.
Timeout ao tentar ir para a primeira página.
Dados raspados: 50 linhas
Dados raspados: 50 linhas
Dados raspados: 50 linhas
Dados raspados: 50 linhas
Dados raspados: 50 linhas
Dados raspados: 50 linhas
Dados raspados: 17 linhas
Indo para a próxima filial...
Dados da filial 'RIGARRSPCAPITAL' salvos em 'C:\Users\leonardo.campos\Documents\Automacao\inventario_filial_RIGARRSPCAPITAL.csv'.
Colunas 'Descrição' e/ou 'Produto' não encontradas nos arquivos.
Iniciando raspagem para a filial: RIGARRRJCAPITAL
Filial 'RIGARRRJCAPITAL' selecionada com sucesso.
Timeout ao tentar ir para a primeira página.
Dados raspados: 50 linhas
Dados raspados: 50 linhas
Dados raspados: 50 linhas
Dados raspados: 50 linhas
Dados raspados: 50 linhas
Dados raspados: 50 linhas
Dados raspados: 15 linhas
Indo para a próxima filial...
Dados da filial 'RIGARRRJCAPITAL' salvos 